In [ ]:
import pip
pip.main(['install','pygame'])

In [1]:
import pygame
from skimage.measure import find_contours
from skimage.draw import rectangle
from skimage.color import rgba2rgb,rgb2gray
from skimage.morphology import binary_erosion,binary_opening, binary_dilation, binary_closing,skeletonize, thin
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np
from skimage.exposure import histogram
from matplotlib.pyplot import bar
from skimage.color import rgb2gray,rgb2hsv

# Convolution:
from scipy.signal import convolve2d
from scipy import fftpack
import math

from skimage.util import random_noise
from skimage.filters import median
from skimage.feature import canny
from skimage.measure import label
%matplotlib inline
%load_ext autoreload
%autoreload 2

pygame 2.1.2 (SDL 2.0.18, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show() 

In [28]:
def countFingers(xa,fingers,xs):
            [x,x1,x2,x3,x4,x5]=xs
            if(xa>x and xa<=x1):
                    fingers[0]=1
            elif(xa>x1 and xa<=x2):
                 fingers[1]=1
            elif(xa>x2 and xa<=x3):
                 fingers[2]=1
            elif(xa>x3 and xa<=x4):
                 fingers[3]=1
            elif(xa>x4 and xa<=x5):
                 fingers[4]=1
            return fingers
def getFingers(img,palm):
    palm=np.where(palm>0.9,1,0)
    psum=np.sum(palm, axis=1)
    y=np.argmax(psum)
    w=np.max(psum)
    i=0
    while(palm[y,i]==0):
        i+=1
    x=i
    f=w/5
    x1=x+f
    x2=x+2*f
    x3=x+3*f
    x4=x+4*f
    x5=x+w
    xs=[x,x1,x2,x3,x4,x5]
    #print(x,w,f)
#img=binary_opening(img)*1
#img=binary_closing(img)*1
    img=binary_erosion(img)
    img=binary_erosion(img)
    img=binary_erosion(img)
    img=binary_dilation(img)*0.99
    contours=find_contours(img,0.9)
    bounding_boxes=[]
    fingers=[0,0,0,0,0]
    for i in range (0,len(contours)):
        xmin=min(contours[:][i][:,1])
        xmax=max(contours[:][i][:,1])
        ymin=min(contours[:][i][:,0])
        ymax=max(contours[:][i][:,0])
        fw=xmax-xmin
        xa=(xmin+xmax)/2
        aspect=(xmax-xmin)/(ymax-ymin)
        if(aspect<=0.4):
            fingers=countFingers(xa,fingers,xs)
        elif(aspect<=0.7):
            xa2=(xmin+xa)/2
            xa3=(xa+xmax)/2
            fingers=countFingers(xa2,fingers,xs)
            fingers=countFingers(xa3,fingers,xs)
        elif(aspect<=0.9):
            t=(xmax-xa)/3
            xa2=xmin+t
            xa3=xa+2*t
            fingers=countFingers(xa,fingers,xs)
            fingers=countFingers(xa2,fingers,xs)
            fingers=countFingers(xa3,fingers,xs)
        elif(xa>x-f and xa<=x1):
            fingers[0]=1
        elif(aspect<=1.5 and xa>x1 and xa<=x5):
            fingers[4]=1
            fingers[3]=1
            fingers[2]=1
            fingers[1]=1
        #print(fingers)
       # bounding_boxes.insert(0,[int(min(contours[:][i][:,1])),int(max(contours[:][i][:,1])),int(min(contours[:][i][:,0])),int(max(contours[:][i][:,0]))])
    #img_gray=img
    #img_with_boxes=img*0
    #print(x,x1,x2,x3,x4,x5)
    #for box in bounding_boxes:
     #   [Xmin, Xmax, Ymin, Ymax] = box
     #   rr, cc = rectangle(start = (Ymin,Xmin), end = (Ymax,Xmax), shape=img_gray.shape)
     #   img_with_boxes[rr, cc] = 1 #set color white
    #show_images(images=[img_with_boxes],titles=['Image'])
    return fingers

In [29]:
def action(sound,s,v,act,p,m):
    if(act==1):
        p=1
        sound.play()
    elif(act==2):
        v+=0.4
        sound.set_volume(v) 
    elif(act==3):
        v-=0.4
        sound.set_volume(v)
    elif(act==4):
        p=1
        sound.stop()
        if(s==m):
            s=1
        else:
            s+=1
        sound=pygame.mixer.Sound(str(s)+'.mp3')
        sound.play()
    elif(act==5):
        p=1
        sound.stop()
        if(s==1):
            s=m
        else:
            s-=1
        sound=pygame.mixer.Sound(str(s)+'.mp3')
        sound.play()
    elif(act==6):
        p=0
        sound.stop()
    return sound,s,v,p
def myApp(img,palm,s,v,sound,p,m):
    fingers=getFingers(img,palm)
    print(fingers)
    act=0
    if(np.sum(fingers)==5 and p==0):
        act=1
    elif(np.sum(fingers)==5 and p==1):
        act=6
    elif((fingers[0] or fingers[1]) and np.sum(fingers)==1):
        act=2
    elif(np.sum(fingers)==0):
        act=3
    elif(fingers[0] and fingers[1] and not(fingers[3]) and not(fingers[4])):
        act=4
    elif((fingers[0] or fingers[1]) and fingers [4] and np.sum(fingers)<=3):
        act=5
    else:
        act=0
    print(act)
    return action(sound,s,v,act,p,m)

In [30]:
pygame.init()
pygame.mixer.init()
s=1
v=0.5
m=2
p=0
images=['31.jpg','21.jpeg','34.jpg','29.jpeg','32.jpg']
palms=['18.png','22.jpeg','33.jpg','30.jpeg','18.png']
act=1
sound=pygame.mixer.Sound(str(s)+'.mp3')
while(act>0):
    act=int(input())
    if(act==0):
        break
    img=rgb2gray(io.imread(images[act-1]))
    palm=rgb2gray(io.imread(palms[act-1]))
    #show_images(images=[img,palm],titles=['fingers','palm'])
    sound,s,v,p=myApp(img,palm,s,v,sound,p,m)
    print(s,v,p)
    

1


<ipython-input-30-69e5f0a60a2b>:16: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  palm=rgb2gray(io.imread(palms[act-1]))


[1, 1, 1, 1, 1]
1
1 0.5 1
1
[1, 1, 1, 1, 1]
6
1 0.5 0
0
